# Outgrowth simulations
Proportional mapping of S and partial synchronization of G1 model

In [ ]:
# required libraries
import numpy as np
from scipy import stats
import pandas as pd
import os

In [ ]:
def cellsCycling(ts,cells,rl):
# a cell divides when it completes its cell cycle,
# meaning that the time remaining to cell division ('tr') reachs zero.
    cells_that_divide = []
    for cell_id in cells.keys():
    # go through each cell
        if not cells[cell_id]['dormant']:
            cells[cell_id]['tc'] += 1 # updates cell cycle position (simulation time dependent)
            if cells[cell_id]['td'] == cells[cell_id]['tc']: # checks if cell cycle is completed 
            # if the cell completes its cell cycle
                cells_that_divide.append(cell_id)
        else:
            if cells[cell_id]['recruited']:
                cells[cell_id]['awakeningDelay'] -= 1
                if cells[cell_id]['awakeningDelay'] == 0:
                    cells[cell_id]['dormant'] = False
        if cells[cell_id]['position'] >= rl and ts < tau and not cells[cell_id]['recruited']:
        # signal pulse at time tau in lambda microns
            cells = cycleShortening(cells,cell_id) # cell recruitment          
    cells = updatePositions(cells,cells_that_divide) # cell pushing mechanism        
    cells = cellsDivision(cells,cells_that_divide) # cell division
    for cell_id in cells.keys():
        cells = phaseIdentifier(cells,cell_id) # identifies in which phase the cell is    
    return cells

In [ ]:
def cycleShortening(cells,cell_id):
# cell cycle shortening implementation
    cycle_position = cells[cell_id]['tc']
    cycle_length = cells[cell_id]['td']
    g1_length = int(cycle_length*long_g1_proportion)
    g1_reduction = int(cycle_length*g1_reduction_proportion)
    s_length = int(cycle_length*long_s_proportion)
    s_reduction = int(cycle_length*s_reduction_proportion)
    g2m_length = int(cycle_length*long_g2m_proportion)
    if 0 <= cycle_position <= g1_reduction:
    # cell in the G1 skip
        # G1 skipping part 1 (partial synchronization implementation part 1)
        cells[cell_id]['tc'] = cycle_position-cycle_position
    elif g1_reduction < cycle_position <= g1_length:
    # cell in the rest of G1
        # G1 skipping part 2 (partial synchronization implementation part 2)
        cells[cell_id]['tc'] = cycle_position-g1_reduction
    elif g1_length < cycle_position <= g1_length+s_length:
    # cell in S phase
        # S mapping (proportional mapping implementation)
        cells[cell_id]['tc'] = int((cycle_position-g1_length)*((s_length-s_reduction)/s_length)+(g1_length-g1_reduction))
    elif g1_length+s_length < cycle_position <= g1_length+s_length+g2m_length+2:
    # cell in G2/M
        cells[cell_id]['tc'] = cycle_position-g1_reduction-s_reduction
    cells[cell_id]['td'] = cycle_length-g1_reduction-s_reduction
    cells[cell_id]['recruited'] = True
    cells[cell_id]['dormant'] = False    
    return cells

In [ ]:
def updatePositions(cells,cells_that_divide):
# cell pushing mechanism implementation
    movements = {}
    for cell_id in cells.keys():
        cell_movement = 0
        for divided_cell in cells_that_divide:
            if cells[cell_id]['position'] >= cells[divided_cell]['position']:
                cell_movement += 1
        movements[cell_id] = cell_movement
    for cell_id in cells.keys():
        cells[cell_id]['position'] = cells[cell_id]['position']+movements[cell_id]*cell_diameter
    return cells

In [ ]:
def cellsDivision(cells,cells_that_divide):
# creates new cells based on mothers properties
    for cell_id in cells_that_divide:
        cells[cell_id]['tc'] = 0
        daughter_id = len(cells)
        if cells[cell_id]['recruited']:
        # daughters of recruited cells are also recruited cells
            cells[cell_id]['td'] = lognormal(short_cycle_mean,short_cycle_std)
            cells[daughter_id] = {'tc':0,
                                  'td':lognormal(short_cycle_mean,short_cycle_std),
                                  'recruited':True,
                                  'position':cells[cell_id]['position']-cell_diameter,
                                  'dormant':False,
                                  'awakeningDelay':0,
                                  'clone':cell_id}
        else:
        # daughters of non-recruited cells are also non-recruited cells 
            cells[cell_id]['td'] = lognormal(long_cycle_mean,long_cycle_std)
            cells[daughter_id] = {'tc':0,
                                  'td':lognormal(long_cycle_mean,long_cycle_std),
                                  'recruited':False,
                                  'position':cells[cell_id]['position']-cell_diameter,
                                  'dormant':False,
                                  'awakeningDelay':0,
                                  'clone':cell_id}
    return cells

In [ ]:
def phaseIdentifier(cells,cell_id):
# identifies the current cell phase 
    cycle_position = cells[cell_id]['tc']
    cycle_length = cells[cell_id]['td']
    if cells[cell_id]['recruited']:         
        g1_length = int(cycle_length*short_g1_proportion)
        s_length = int(cycle_length*short_s_proportion)
        g2m_length = int(cycle_length*short_g2m_proportion)
        if 0 <= cycle_position <= g1_length:
            # G1 phase
            cells[cell_id]['phase'] = "G1"
        elif g1_length < cycle_position <= g1_length+s_length:
            # S phase
            cells[cell_id]['phase'] = "S"
        elif g1_length+s_length < cycle_position <= g1_length+s_length+g2m_length+2:
            # G2/M phase
            cells[cell_id]['phase'] = "G2/M"
    else:      
        g1_length = int(cycle_length*long_g1_proportion)
        s_length = int(cycle_length*long_s_proportion)
        g2m_length = int(cycle_length*long_g2m_proportion)
        if 0 <= cycle_position <= g1_length:
            # G1 phase
            cells[cell_id]['phase'] = "G1"
        elif g1_length < cycle_position <= g1_length+s_length:
            # S phase
            cells[cell_id]['phase'] = "S"
        elif g1_length+s_length < cycle_position <= g1_length+s_length+g2m_length+2:
            # G2/M phase
            cells[cell_id]['phase'] = "G2/M"
    if cells[cell_id]['dormant'] or cells[cell_id]['awakeningDelay'] > 0:
        cells[cell_id]['phase'] = "G0"
    return cells

In [ ]:
def tc_distribution(td):
    x = np.arange(0,td+1)
    fn = 2**(1-x*p/td)
    fn /= fn.sum() # normalization
    tc = np.random.choice(x, p=fn)
    return tc

In [ ]:
def lognormal(mu_x,dt_x,size=1,integer=True):
# Draw one value (or more if size > 1) from a discretized lognormal distribution
    mu = np.log(mu_x**2/np.sqrt(mu_x**2+dt_x**2))
    sigma = np.sqrt(np.log(1+dt_x**2/mu_x**2))
    shape = sigma # Scipy's shape parameter
    scale = np.exp(mu) # Scipy's scale parameter
    distribution = stats.lognorm.rvs(scale=scale,s=shape,size=size)
    if len(distribution) == 1:
        if integer:
            return int(distribution[0])
        else:
            return distribution[0]
    else:
        return distribution

In [ ]:
def run():
# simulation run
    
    # initial conditions
    cells = {}
    for cell_id in range(0,n0):
        cell_key = cell_id
        td = lognormal(long_cycle_mean,long_cycle_std)
        tc = tc_distribution(td)
        cells[cell_key] = {'td':td, # cell cycle length
                           'tc':tc, # cell cycle position
                           'position':(cell_key+1-n0)*cell_diameter,
                           'recruited':False,
                           'dormant':False,
                           'awakeningDelay':0,
                           'clone':n0-cell_key}
        
    g0_cells_number = int(n0*g0_prop)
    cells_df = pd.DataFrame.from_dict(cells, orient='index')
    g0_cells = cells_df[cells_df['tc'] <= long_g1].sample(g0_cells_number).index
    cells_df.loc[g0_cells,'dormant'] = True
    cells_df.loc[g0_cells,'awakeningDelay'] = awakeningDelay
    cells = cells_df.to_dict(orient='index')
    
    # time iteration
    simulation = {} # empty simulation output 
    ts = 0 # simulation time = 0
    for ts in range(0,steps):
        signal_pos = ts*(-l/tau)
        cells = cellsCycling(ts,cells,signal_pos)
        cells_df = pd.DataFrame.from_dict(cells, orient='index')
        simulation[ts] = cells_df       
    return simulation

In [ ]:
# run parameters    
n0_mean,n0_std = 196,2 # n0 mean and standar deviation
l_mean,l_std = 828,30      # lambda mean and standar deviation
tau_mean = 2

steps = 1+24*8           # number of steps (in hours)
np.random.seed(0) 
seeds_number = 100       # number of simulations
    
# constants
cell_diameter = 13.2 # cell diameter
long_g1 = 152            # G1 length in long cycle
long_s = 179             # S length in long cycle
short_g1 = 22            # G1 length in short cycle
short_s = 88             # S length in short cycle                
long_g2m = short_g2m = 9 # G2/M length in both, long and short cycle
long_cycle_mean = long_g1+long_s+long_g2m      # long cell cycle mean
long_cycle_std = 32                            # long cell cycle standar deviation
short_cycle_mean = short_g1+short_s+short_g2m  # short cell cycle mean
short_cycle_std = 10                           # short cell cycle standar deviation
long_g1_proportion = long_g1/long_cycle_mean                 # G1 proportion in the long cell cycle
long_s_proportion = long_s/long_cycle_mean                   # S proportion in the long cell cycle
long_g2m_proportion = long_g2m/long_cycle_mean               # G2/M proportion in the long cell cycle
short_g1_proportion = short_g1/short_cycle_mean              # G1 proportion in the short cell cycle
short_s_proportion = short_s/short_cycle_mean                # S proportion in the short cell cycle
short_g2m_proportion = short_g2m/short_cycle_mean            # G2/M proportion in the short cell cycle 
g1_reduction_proportion = (long_g1-short_g1)/long_cycle_mean # proportion of G1 reduction in the long cell cycle
s_reduction_proportion = (long_s-short_s)/long_cycle_mean    # proportion of S reduction in the long cell cycle
g0_prop = 0.12      # G0 cells proportion
awakeningDelay = 72 # G0 cells activation delay
p = 2 # tc_distribution perturbation

# directory name
root = '../main/simulations/'
model = 'outgrowth/'
parameters = 'n0='+str(n0_mean)+'\n'+'l='+str(l_mean)+'\n'+'tau='+str(tau_mean)+'/'
path = root+model+parameters
if not os.path.isdir(path):
    os.makedirs(path)

# simulations
for seed in range(1,seeds_number+1):
    print('Runing seed number:',seed, end="\r", flush=True)
    
    # parameters drawing
    n0 = int(np.random.normal(n0_mean,n0_std))
    l = int(np.random.normal(l_mean,l_std))
    tau = tau_mean
    
    # simulation run
    simulation = run()
    
    # output file for each seed
    parameters = 'seed='+str(seed)+'_n0='+str(n0)+'_'+'l='+str(l)+'_'+'tau='+str(tau)
    data = pd.concat(simulation, names=['time','id'])
    outfile = open(path+parameters+'.csv', 'a')
    data.to_csv(outfile, sep=',')
    outfile.close()